Outline of simulations:
- generate all possible resource preference lists
    
- generate growth rates for each resource for each species (draw them from a normal distribution, to avoid super-bug and extreme specialists)
- pick resource concentrations (start with equal case 1:1, then test 2:1 etc.)
- perform species invasions from the pool until the uninvadable state is reached via dynamic simulations
- keep track of invasion events, so we'll be able to plot diversity w/ time, complementarity w/ time, etc.

How to do serial dilution computation - perform stepwise estimation for resource usage:
- start with $B_0$ and $C_0$, get equations for each resource, solve for T
- compare T of resource depletion
- pick the smallest time, switch resources
- update bacterial abundances and resource concentrations
- continue until end of dilution cycle reached/all resources depleted
- every bacteria, with population below the threshold (e.g. $10^{-5}$) is doomed to be extinct
- the steady state should be reached in 5-10 dilution cycles
- make sure that any leftovers of the resources, that are unused by the end of the cycle are added to the new one with the same dilution factor as bacterial abundances (it's important for the low dilution case)

List of questions
- what is the optimal diauxic strategy?
- we expect to get a set of microbes with a complementary growth strategies in the end. Is it true?
- is it true that unequal resource supply will change the winning strategy?


In [1]:
# initialization

import numpy as np
import math
import random
import itertools
from scipy.optimize import root
import copy

# set resource number and amount
Nr = 2
Res = [1.0, 1.0]

# generate species pool: growth rate from gaussian; preference list; invasion order
# growth rates
Nb = math.factorial(Nr)
g_mean, g_var = 2, 0.5
growth_rate_list = np.random.normal(g_mean, math.sqrt(g_var), (Nb, Nr))
filt = np.heaviside(growth_rate_list, 0)
growth_rate_list = filt*growth_rate_list
#preference list
preference_list = list(itertools.permutations(range(Nr), Nr))
b0 = 1e-3 # density of bug when introduced/initial
b_threshold = 1e-7 # extinction density
c_threshold = 1e-9 # concentration threshold
# invasion order
invasion_list = list(range(Nb))
random.shuffle(invasion_list)
# yields
yields_list = 0.5*np.ones([Nb, Nr]) # might need to be modified later?

# dilution parameters 
D = 1e-3
tau = 0 # lag
T_dilute = 10 # time interval between dilutions
dilute_to_steady = 8 # #(dilution) between invasions

In [2]:
def dilute(system): 
    global t_system
    t_switch = 0
    while t_switch < T_dilute:
        #list of resource in use for each consumer
        use = [0 for i in range(Nb)]
        # list of consumers for each resource
        consumer = [[] for i in range(Nr)]
        for i in range(Nb):
            # if all are depleted, bug still uses its least preferred nutrient
            while system['res_available'][preference_list[i][use[i]]] < 1 and use[i] < Nr - 1:
                use[i] = use[i] + 1
            if system['bug_available'][i] > 0:
                consumer[preference_list[i][use[i]]].append(i)
        # find the earliest depleted resource
        t_dep = T_dilute - t_switch
        for i in range(Nr):
            if system['res_available'][i] > 0:
                def remain(t):  
                    # S = c - sum(B0(e^gt-1)/Y)                
                    return system['res_concentration'][i] - sum([system['bug_density'][j]*(math.exp(t*growth_rate_list[j][i]) - 1)/yields_list[j][i] for j in consumer[i]])
                # a little bit tricky here...
                t_i = root(remain, T_dilute + 1).x[0]
                if t_i < t_dep:
                    t_dep = t_i
        # update the system according to this t_dep
        t_switch = t_switch + t_dep
        temp_bug_density = [i for i in system['bug_density']]
        for i in range(Nb):
            if system['res_available'][preference_list[i][use[i]]] > 0:
                temp_bug_density[i] = system['bug_density'][i]*math.exp(growth_rate_list[i][preference_list[i][use[i]]]*t_dep)
        for i in range(Nr):
            if system['res_available'][i] > 0:
                system['res_concentration'][i] = system['res_concentration'][i] - sum([system['bug_density'][j]*(math.exp(t_dep*growth_rate_list[j][i]) - 1)/yields_list[j][i] for j in consumer[i]])
                if system['res_concentration'][i] < c_threshold: 
                    system['res_available'][i] = 0
        for i in range(Nb):
            system['bug_density'][i] = temp_bug_density[i]
        print(system)
    system_copy = copy.deepcopy(system)
    t_system.append(system_copy)
    print("diluted once")
    return system
        
    

def invade(system, bug):
    print("one invasion")
    # (starting from a new flask)
    # then add invasive species
    system['bug_available'][bug] = 1
    system['bug_density'][bug] = b0
    # then dilute till steady
    for i in range(dilute_to_steady):
        system = dilute(system)
        # move to a new flask
        system['bug_density'] = [(i*D > b_threshold)*i*D for i in system['bug_density']]
        system['bug_available'] = [1*(system['bug_density'][i] > b_threshold) for i in range(Nb)]
        system['res_concentration'] = [(system['res_concentration'][i]*D + Res[i]) / (D + 1) for i in range(Nr)]
        system['res_available'] = [1*(system['res_concentration'][i] > c_threshold) for i in range(Nr)]
        
    ext_list = [i for i, v in enumerate(system['bug_available']) if v==0]
    return system, ext_list
    
def round_robin_invade(system, ext_list):
    for bug in ext_list:
        system, new_ext_list = invade(system, bug)
    return system, new_ext_list

In [3]:
# simulation
Res_copy = [i for i in Res]
system = {'res_available': np.heaviside(Res, 0), 'res_concentration': Res_copy, 'bug_available': [0 for i in range(Nb)], 'bug_density': [0 for i in range(Nb)]} 
t_system = [system] # snapshots of systems at the end of each dilution
# ext_list: list of bugs not in the community
old_ext_list = invasion_list
new_ext_list = []
while new_ext_list != old_ext_list:
    print(old_ext_list)
    system, new_ext_list = round_robin_invade(system, old_ext_list)
    old_ext_list = new_ext_list

[1, 0]
one invasion
{'res_available': array([1., 0.]), 'res_concentration': [1.0, -6.372680161348399e-14], 'bug_available': [0, 1], 'bug_density': [0.0, 0.5010000000000319]}
{'res_available': array([0., 0.]), 'res_concentration': [1.1102230246251565e-16, -6.372680161348399e-14], 'bug_available': [0, 1], 'bug_density': [0.0, 1.0010000000000319]}
{'res_available': array([0., 0.]), 'res_concentration': [1.1102230246251565e-16, -6.372680161348399e-14], 'bug_available': [0, 1], 'bug_density': [0.0, 1.0010000000000319]}
diluted once
{'res_available': [1, 0], 'res_concentration': [0.9990009990009991, -6.550315845288424e-14], 'bug_available': [0, 1], 'bug_density': [0.0, 0.5005014995005322]}
{'res_available': [0, 0], 'res_concentration': [1.1102230246251565e-16, -6.550315845288424e-14], 'bug_available': [0, 1], 'bug_density': [0.0, 1.0000019990010318]}
{'res_available': [0, 0], 'res_concentration': [1.1102230246251565e-16, -6.550315845288424e-14], 'bug_available': [0, 1], 'bug_density': [0.0, 

In [4]:
t_system

[{'res_available': [1, 1],
  'res_concentration': [0.9990009990009991, 0.9990009990009991],
  'bug_available': [1, 0],
  'bug_density': [0.0010000010000010104, 0.0]},
 {'res_available': array([0., 0.]),
  'res_concentration': [1.1102230246251565e-16, -6.372680161348399e-14],
  'bug_available': [0, 1],
  'bug_density': [0.0, 1.0010000000000319]},
 {'res_available': [0, 0],
  'res_concentration': [1.1102230246251565e-16, -6.550315845288424e-14],
  'bug_available': [0, 1],
  'bug_density': [0.0, 1.0000019990010318]},
 {'res_available': [0, 0],
  'res_concentration': [1.1102230246251565e-16, -6.517009154549669e-14],
  'bug_available': [0, 1],
  'bug_density': [0.0, 1.0000010010000326]},
 {'res_available': [0, 0],
  'res_concentration': [2.220446049250313e-16, -6.483702463810914e-14],
  'bug_available': [0, 1],
  'bug_density': [0.0, 1.0000010000020314]},
 {'res_available': [0, 0],
  'res_concentration': [0.0, -6.561418075534675e-14],
  'bug_available': [0, 1],
  'bug_density': [0.0, 1.0000